Obrada slike u medicini
Laboratorijska vežba 5:
Obrada rendgenske slike: Logaritamska kompresija opsega
Potrebne biblioteke: cv2, numpy, matplotlib.pyplot
1. Sirova slika
1.1 Učitati sirovu sliku Ro_01.fxd koristeći funkciju read_raw iz modula osum.py.
1.2 U kom opsegu se nalaze vrednosti slike? Koji je tip slike? Koje su dimenzije slike?
1.3 Prikazati sliku. Koji modalitet je u pitanju i šta se vidi na slici?
1.4 Napraviti histogram slike u granicama od 0 do 16384 sa korakom 8.
1.5 Prikazati histogram. Kako izgleda raspodela i kako se to odražava na izgled snimka?
2. Logaritamska kompresija opsega
2.1 U modulu osum.py napraviti funkciju log_LUT u kojoj se pravi look-up tabela sa
logaritamskom kompresijom opsega. Ulazni parametri funkcije su ulazni opseg, izlazni
opseg i tolerancija do koje se komprimuje linearno.
2.2 Napraviti LUT za logaritamsku kompresiju opsega tako da se ulazni opseg od 16384
pretvori u opseg 4096 sa tolerancijom 0.001. Primeniti LUT na ulaznu sliku.
2.3 Prikazati sliku nakon logaritamske kompresije opsega. Koji je efekat ove
transformacije?
2.4 Napraviti histogram slike nakon primenjene transformacije i na istom grafiku prikazati
histogram pre i nakon transformacije. Kako se razlikuju raspodele?
2.5 Ponoviti korak 2.2 koristeći toleranciju od 0.1. Prikazati sliku. Zašto se vidi manje
struktura?

In [ ]:
import cv2
import osum
import matplotlib.pyplot as plt
import numpy as np

# 1.1
[im, DetInfo, hdr] = osum.read_raw('Ro_01.fxd')

# 1.3
cv2.imshow('Ulazna slika', im/im.max())
cv2.waitKey(0)
# cv2.destroyAllWindows()

# 1.4
g = np.arange(0, 16385, 8)
h1 = np.histogram(im, g)

# 1.5
plt.figure()
plt.plot(g[:-1], h1[0])
plt.show()

# 2.2
lut = osum.log_LUT(16384, 4096, 0.001)
# lut = osum.log_LUT(16384, 4096, 0.1) # 2.5
im_log = lut[im]

# 2.3
cv2.imshow('Slika nakon log kompresije opsega', im_log/im_log.max())
cv2.waitKey(0)
cv2.destroyAllWindows()

# 2.4
h2 = np.histogram(im_log, g)

plt.figure()
plt.plot(g[:-1], h1[0])
plt.plot(g[:-1], h2[0], 'r')
plt.legend(['original', 'log kompresija'])
plt.show()
